In [1]:
import os
import sys

PARENT_DIRNAME = os.path.expanduser("~/image-processing-project/")
IMAGE_DIR = os.path.join(PARENT_DIRNAME, "data/img_align_celeba/")
STORAGE_DATA_DIRNAME = os.path.join(PARENT_DIRNAME, "fine_tuning/data_for_fine_tuning")
MODEL_DIR = os.path.join(PARENT_DIRNAME, "fine_tuning/models")

sys.path.append(PARENT_DIRNAME)

In [2]:
import numpy as np
import json
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
from fine_tuning.triplet_process import GenerateTriplets
from fine_tuning.drop_out import RandomDropout
from fine_tuning.triplet_method import TripletDataset, QueryDataset, collate_fn

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
IMAGE_SIZE = 218
BATCH_SIZE = [1, 32, 64]
NUM_EPOCHS = [5, 10, 15]
NUM_WORKERS = 4
LEARNING_RATE = 0.001
TOP_K = 5

In [8]:
train_triplets, test_queries, test_galleries = GenerateTriplets()

Number of train triplets: 149366
Number of test query triplets: 2020
Number of test gallery triplets: 37253


In [9]:
# save the triplets to a json file
with open(os.path.join(STORAGE_DATA_DIRNAME, "train_triplets.json"), "w") as f:
    json.dump(train_triplets, f)

with open(os.path.join(STORAGE_DATA_DIRNAME, "test_queries.json"), "w") as f:
    json.dump(test_queries, f)

with open(os.path.join(STORAGE_DATA_DIRNAME, "test_galleries.json"), "w") as f:
    json.dump(test_galleries, f)

In [5]:
# load the triplets from the json file
with open(os.path.join(STORAGE_DATA_DIRNAME, "train_triplets.json"), "r") as f:
    train_triplets = json.load(f)

with open(os.path.join(STORAGE_DATA_DIRNAME, "test_queries.json"), "r") as f:
    test_queries = json.load(f)

with open(os.path.join(STORAGE_DATA_DIRNAME, "test_galleries.json"), "r") as f:
    test_galleries = json.load(f)

In [6]:
train_transforms = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value='random'),
    transforms.RandomApply([RandomDropout(p=0.3)], p=0.3)
])

test_transforms = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [7]:
train_dataset = TripletDataset(
    image_dir=IMAGE_DIR,
    train_triplets=train_triplets,
    transform=train_transforms
)

query_dataset = QueryDataset(
    image_dir=IMAGE_DIR,
    query_triplets=test_queries,
    transform=test_transforms
)

gallery_dataset = QueryDataset(
    image_dir=IMAGE_DIR,
    query_triplets=test_galleries,
    transform=test_transforms
)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE[2], shuffle=True, num_workers=NUM_WORKERS, collate_fn=collate_fn)
query_loader = DataLoader(query_dataset, batch_size=BATCH_SIZE[0], shuffle=False, num_workers=NUM_WORKERS, collate_fn=collate_fn)
gallery_loader = DataLoader(gallery_dataset, batch_size=BATCH_SIZE[1], shuffle=False, num_workers=NUM_WORKERS, collate_fn=collate_fn)

In [8]:
torch.save(train_loader, os.path.join(STORAGE_DATA_DIRNAME, "train_loader.pth"))
torch.save(query_loader, os.path.join(STORAGE_DATA_DIRNAME, "query_loader.pth"))
torch.save(gallery_loader, os.path.join(STORAGE_DATA_DIRNAME, "gallery_loader.pth"))